# Classify a dynamic length sequence

THIS NOTEBOOK IS _OUT DATED_. PLEASE SEE `Classify.py` FOR LATEST.

In [ ]:
#from core import generate_sample_group
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed
from keras.utils import to_categorical
import numpy as np
import random

In [2]:
def generate_num_nonzero(num_samples, lognorm_mean=3, lognorm_sigm=1):
    # Select number of nonzero terms in spectroscopy sample
    return np.random.lognormal(lognorm_mean, lognorm_sigm, num_samples).astype(int)


def generate_mass_samples(num_nonzero, num_samples,
                          low=0, high=300, mass_std=0.001,
                          intensity_spread_factor=1.0, intensity_std=0.01):
    # Generate the location of the num_nonzero components
    #intensity_locs = np.random.dirichlet(np.ones(num_nonzero)*float(intensity_spread_factor))
    counts = np.random.uniform(0, 1, num_nonzero)
    intensity_locs = counts / counts.sum()

    samples = []
    for i in range(num_nonzero):
        loc = np.random.uniform(low, high)
        mass_samples = np.random.normal(loc, mass_std, num_samples)
        intensity_samples = np.random.normal(intensity_locs[i], intensity_std, num_samples)
        samples.append(np.column_stack((mass_samples, intensity_samples)))
    
    samples = np.asarray(list(zip(*samples)))
    # ToDo: Improve. This is a weak fix.
    samples[samples < 0] = 0
    return samples

def generate_sample_group(num_samples, num_nonzero=None, lognorm_mean=3, lognorm_sigm=1,
                         low=0, high=300, mass_std=0.001,
                         intensity_spread_factor=1.0, intensity_std=0.01):
    if isinstance(num_nonzero, type(None)):
        num_nonzero = generate_num_nonzero(1, lognorm_mean=3, lognorm_sigm=1)[0]
    samples = generate_mass_samples(num_nonzero, num_samples,
                              low=low, high=high, mass_std=mass_std,
                              intensity_spread_factor=intensity_spread_factor, intensity_std=intensity_std)
    return samples, num_nonzero


def generate_all_samples(ngroups=100, nsamples_in_group=10, **kwargs):
    """Generates samples for multiple test groups.
    
    Returns
    -------
    X : `ngroup` size list of arrays: (nsamples_in_group, random_num_nonzero, 2)
    y : `ngroup`*`nsamples_in_group` size list of strings
    """
    X = []
    y = []
    lengths = []
    for group_i in range(ngroups):
        samples, num_nonzero = generate_sample_group(nsamples_in_group, **kwargs)       
        X.extend(samples)
        y.extend([group_i]*nsamples_in_group)
        lengths.extend([num_nonzero]*nsamples_in_group)
    return X, np.array(y), np.array(lengths)

In [3]:
nsamples_in_group = 10
ngroups = 100
X, y, lengths = generate_all_samples(ngroups=ngroups, nsamples_in_group=nsamples_in_group, num_nonzero=None)
y = to_categorical(y)

print("X:", len(X), "First sample in X:", X[0].shape)
print("y:", y.shape)

f"X has {len(X)} samples, built by {ngroups} groups with {nsamples_in_group} samples each. "+\
f"The first group is of size {X[0].shape}, where the first index is the number of nonzero "+\
f"entries and 2 defines the ordered pair (mass and intensity). Y has {len(y)} definitions "+\
f"each of size {len(y[0])} to record the one-hot encoded vector."

X: 990 First sample in X: (94, 2)
y: (1000, 100)


'X has 990 samples, built by 100 groups with 10 samples each. The first group is of size (94, 2), where the first index is the number of nonzero entries and 2 defines the ordered pair (mass and intensity). Y has 1000 definitions each of size 100 to record the one-hot encoded vector.'

In [4]:
# Shuffle
Xyl = list(zip(X, y, lengths))
random.shuffle(Xyl)
X, y, lengths = zip(*Xyl)

# Split into train-test splits
train_pct = 0.9
train_split = int(train_pct * len(y))

trainX, trainy, trainLengths = X[:train_split], y[:train_split], lengths[:train_split]
testX, testy = X[train_split:], y[train_split:]

In [5]:
import pandas as pd
pd.Series(trainLengths).value_counts()

7      71
18     38
15     34
11     32
4      29
       ..
120     8
133     8
263     8
28      7
5       7
Length: 62, dtype: int64

In [22]:
model = Sequential()

model.add(LSTM(32, return_sequences=True, input_shape=(None, 2)))
model.add(LSTM(8, return_sequences=False))
model.add(Dense(125, activation='sigmoid'))

print(model.summary())
model.compile(loss='categorical_crossentropy',
              optimizer='adam')

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, None, 32)          4480      
_________________________________________________________________
lstm_7 (LSTM)                (None, 8)                 1312      
_________________________________________________________________
dense_3 (Dense)              (None, 125)               1125      
Total params: 6,917
Trainable params: 6,917
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
# def train_generator():
#     global trainX, trainy, trainLengths
#     trainyarr = np.array(trainy)
#     while True:
#         sequence_length = np.random.choice(trainLengths)
#         #TODO: is seeing same data multiple times
#         mask = np.array(trainLengths)==sequence_length
#         #x_train = trainX[mask, :, :]
#         #y_train = trainy[mask, :]
#         arrs = []
#         for b,arr in zip(mask,trainX):
#             if b:
#                 arrs.append(arr)
                
#         #x_train = np.asarray(trainX)[mask]
#         x_train = np.asarray(arrs)
# #         for i in range(len(x_train)):
# #             print(x_train[i].shape, type(x_train[i]))
# #         print(x_train.shape)
#         #print(x_train.shape, x_train[0].shape, x_train[-1].shape)
#         y_train = trainyarr[mask]
#         print(x_train.shape, y_train.shape, type(x_train), type(y_train))
        
#         yield x_train, y_train

# def train_generator():
#     while True:
#         sequence_length = np.random.randint(10, 100)
#         x_train = np.random.random((1000, sequence_length, 5))
#         # y_train will depend on past 5 timesteps of x
#         y_train = x_train[:, :, 0]
#         for i in range(1, 5):
#             y_train[:, i:] += x_train[:, :-i, i]
#         y_train = to_categorical(y_train > 2.5)
#         print(x_train.shape, y_train.shape)
#         yield x_train, y_train

testX, testY = [], []
def train_generator():
    global testX, testY

    while True:
        sequence_length = generate_num_nonzero(1, lognorm_mean=3, lognorm_sigm=1)[0]
        sequence_length=3
        generate_num_nonzero
        nsamples_in_group = 10
        ngroups = 125
        X, y, lengths = generate_all_samples(ngroups=ngroups,
                                             nsamples_in_group=nsamples_in_group,
                                             num_nonzero=sequence_length)
        X = np.asarray(X)
        y = to_categorical(y)

        # Shuffle
        Xyl = list(zip(X, y, lengths))
        random.shuffle(Xyl)
        X, y, lengths = zip(*Xyl)
        X=np.asarray(X)
        y=np.asarray(y)

        # Split into train-test splits
        train_pct = 0.9
        train_split = 1000#int(train_pct * len(y))

        trainX, trainy, trainLengths = X[:train_split], y[:train_split], lengths[:train_split]
        testX.extend(X[train_split:])
        testY.extend(y[train_split:])
        print(trainX.shape, trainy.shape)
        yield trainX, trainy


model.fit_generator(train_generator(), steps_per_epoch=10, epochs=10, verbose=1)
#model.fit(X,y)

(1000, 3, 2) (1000, 125)
Epoch 1/10
(1000, 3, 2) (1000, 125)


ValueError: Creating variables on a non-first call to a function decorated with tf.function.